In [ ]:
! mkdir ~/.kaggle

In [ ]:
import torch
device = 'gpu' if torch.cuda.is_available() else 'cpu'
device

'gpu'

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d constantinwerner/multilingual-ner-dataset

  0% 0.00/818k [00:00<?, ?B/s]
100% 818k/818k [00:00<00:00, 114MB/s]


In [ ]:
! unzip multilingual-ner-dataset.zip

Archive:  multilingual-ner-dataset.zip
  inflating: ger.tsv                 
  inflating: gre.tsv                 
  inflating: jap.tsv                 
  inflating: rus.tsv                 
  inflating: tur.tsv                 


In [ ]:
from gensim.models.fasttext import FastText
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import numpy as np

реккурентная сетка

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim,  output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim


        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        # batch_dim = number of samples per batch
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True, nonlinearity='relu')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
       # Initialize hidden state with zeros
        hidden_state = torch.zeros(1, 1, self.hidden_dim).to(device)
        # Pass the input sequence through the RNN layer
        rnn_output, hidden_state = self.rnn(x, hidden_state)
        # Reshape the output to be of shape (batch_size * sequence_length, hidden_size)
        rnn_output = rnn_output.contiguous().view(-1, self.hidden_dim)
        # Pass the RNN output through the fully connected layer to get the predicted tags
        predicted_tags = self.fc(rnn_output)

        return predicted_tags

красиво делаем данные


In [ ]:
data = []
labels = []
with open("ger.tsv", "r", encoding="utf-8") as f:
    for line in f:
        tokens = list(line.strip().split("\t"))
        data.append(tokens[0])
        labels.append(tokens[-1])

In [ ]:
print(data)
print(labels)

['1.', 'Auf', 'die', 'Revision', 'des', 'Angeklagten', 'wird', 'das', 'Urteil', 'der', 'auswärtigen', 'großen', 'Strafkammer', 'des', 'Landgerichts', 'Kleve', 'in', 'Moers', 'vom', '12.', 'Oktober', '2017', '', 'a', ')', 'mit', 'den', 'zugehörigen', 'Feststellungen', 'aufgehoben', ',', 'soweit', 'von', 'einer', 'Unterbringung', 'in', 'einer', 'Entziehungsanstalt', 'abgesehen', 'worden', 'ist', ',', '', 'b', ')', 'im', 'Ausspruch', 'über', 'die', 'Sperre', 'für', 'die', 'Erteilung', 'einer', 'Fahrerlaubnis', 'dahin', 'neu', 'gefasst', ',', 'dass', 'die', 'Verwaltungsbehörde', 'angewiesen', 'wird', ',', 'dem', 'Angeklagten', 'vor', 'Ablauf', 'von', 'einem', 'Jahr', 'keine', 'neue', 'Fahrerlaubnis', 'zu', 'erteilen', 'und', 'ihm', 'vor', 'Ablauf', 'dieser', 'Zeit', 'das', 'Recht', ',', 'von', 'einer', 'ausländischen', 'Fahrerlaubnis', 'im', 'Inland', 'Gebrauch', 'zu', 'machen', ',', 'nicht', 'wieder', 'zu', 'erteilen', '.', '', 'Im', 'Umfang', 'der', 'Aufhebung', 'wird', 'die', 'Sache', '

смотрим классы

In [ ]:
label_unique = list(set(labels))
label_unique.remove('')
label_unique

['B-ORDINAL',
 'B-PERSON',
 'I-DATE',
 'B-ORG',
 'I-PERSON',
 'I-ORG',
 'B-LAW',
 'I-LAW',
 'I-PRODUCT',
 'B-CARDINAL',
 'I-CARDINAL',
 'B-PRODUCT',
 'O',
 'I-GPE',
 'B-GPE',
 'B-DATE']

делаем эмбединги

In [ ]:
model = FastText(sentences=data, window=5, min_count=1, workers=4, sg=1)

In [ ]:
embedded_input = [model.wv.get_vector(word) for word in data]

In [ ]:
targets = torch.Tensor(np.array(embedded_input))
labels = LabelEncoder.fit_transform(targets, labels)

In [ ]:
targets = list(zip(targets, labels))

гиперпараметры

In [ ]:
input_size = 100
hidden_size = 64
output_size = len(label_unique)+1
num_epochs = 100
batch_size = 32
wd = 0.0008
learning_rate = 0.001
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.8 MB/s eta 0:00:00


In [ ]:
from torchmetrics.classification import MulticlassAccuracy
nn_model = RNNModel(input_size, hidden_size, output_size)
nn_model = nn_model.to(device)
dataloader = DataLoader(targets, batch_size=batch_size , shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(nn_model.parameters(), lr=learning_rate, weight_decay=wd)
acc = MulticlassAccuracy(num_classes=output_size).to(device)

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    accuracy = 0
    for batch, gt in dataloader:
        batch, gt = batch.to(device), gt.to(device)
        optimizer.zero_grad()
        batch = batch.reshape(1,-1,100)
        outputs = nn_model(batch)
        loss = criterion(outputs.reshape(-1, output_size), gt.reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        accuracy += acc(outputs, gt).item()
    #if (epoch%10 == 0):
    print(f'Epoch {epoch+1}: Loss {total_loss/len(dataloader):.4f}')
    print('accuracy: ', accuracy/len(dataloader))

Epoch 1: Loss 0.5627
accuracy:  0.36860989308141606
Epoch 2: Loss 0.5225
accuracy:  0.38726657240794365
Epoch 3: Loss 0.5192
accuracy:  0.38878218360148675
Epoch 4: Loss 0.5161
accuracy:  0.393149824499215
Epoch 5: Loss 0.5103
accuracy:  0.38963505529575293
Epoch 6: Loss 0.5002
accuracy:  0.39119622006119265
Epoch 7: Loss 0.4867
accuracy:  0.38817784341040107
Epoch 8: Loss 0.4755
accuracy:  0.3922325348819467
Epoch 9: Loss 0.4664
accuracy:  0.40183948374657086
Epoch 10: Loss 0.4597
accuracy:  0.41575307304896497
Epoch 11: Loss 0.4532
accuracy:  0.4189613156350527
Epoch 12: Loss 0.4470
accuracy:  0.4221944680948366
Epoch 13: Loss 0.4416
accuracy:  0.4266928202265443
Epoch 14: Loss 0.4358
accuracy:  0.4324386305262216
Epoch 15: Loss 0.4305
accuracy:  0.4420050847873124
Epoch 16: Loss 0.4254
accuracy:  0.4491213657621689
Epoch 17: Loss 0.4216
accuracy:  0.4564272046137448
Epoch 18: Loss 0.4183
accuracy:  0.4648746359735309
Epoch 19: Loss 0.4148
accuracy:  0.4672134669083241
Epoch 20: Loss